1. Formulate this problem as an integer programming problem and solve it using a solver. Report the solution (the
number of crews and assigned tours) and the cost.

In [1]:
!pip install -q pyomo
!apt-get install -y -qq glpk-utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.4 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 121753 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected pac

In [3]:
from pyomo.environ import *

Let us start with defining our variables, our crews (A,B..), I represent as numbers in i from 0 to 8, let our variable $y_i$ be:

$y_i \begin{cases}
1 & \text{if $i^{th}$ crew is choosen} \\
0 & \text{otherwise}
\end{cases}
$

Since, if we choose the crew $i$ then we can cover a flights mentioned in the sequence. Also let us assume the cost for each crew taken  as $c$ where $\textbf{c[i]}$ will represent the cost of taking crew $i$.


Now let us represent the flights taken by crews. I take variable $a_{ij}$ representing as 1 if flight $i$ is handled by crew $j$.

Let us represent a varible $a_{ij}$ as:
$a_{ij} \begin{cases} 1 & \text{flight i is covered by crew j} \\
0 & \text{otherwise} \end{cases}$

Then our integer program will become:

Objective: $$ \min \sum_{i=0}^{8}c_iy_i $$
Such that : we need to cover each flight atleast once, hence,

$$ \sum_{j = 0}^{8}a_{ij}y_j \geq 1 \ \ \ ∀ \ i = 0,2,...8$$

The objective is to choose as minimum crews as possible, such that for each flight i is covered atleast once. This is ensured as, for each flight $i$ atleast one crew is choosen.

In [4]:
import numpy as np

In [5]:
#creating incidence matrix
A = np.array([[1, 0, 0, 1, 1, 0, 0, 0, 0],
              [0, 1, 0, 0, 0, 1, 0, 1, 0],
              [0, 0, 1, 0, 0, 0, 1, 0, 1],
              [1, 0, 0, 0, 0, 1, 0, 1, 1],
              [0, 1, 0, 0, 0, 0, 0, 0, 1],
              [0, 0, 1, 0, 0, 0, 1, 0, 0],
              [0, 0, 1, 0, 0, 0, 1, 0, 0],
              [1, 0, 0, 0, 0, 1, 0, 1, 0],
              [0, 1, 1, 1, 0, 0, 0, 0, 0],
              [1, 0, 0, 0, 0, 1, 0, 1, 0],
              [0, 0, 0, 1, 1, 0, 1, 0, 1],
              [0, 1, 0, 0, 0, 0, 0, 0, 1],
              [0, 0, 0, 0, 1, 0, 1, 0, 0],
              [0, 0, 0, 1, 0, 1, 0, 1, 0]])

c = np.array([45,45,45,45,35,55,55,60,60])


tours = ['Mumbai to Chennai','Mumbai to Guwahati','Mumbai to Delhi','Delhi to Mumbai','Delhi to Chennai','Delhi to Kolkata','Kolkata to Chennai','Kolkata to Delhi','Chennai to Mumbai','Chennai to Kolkata','Chennai to Guwahati','Guwahati to Delhi','Guwahati to Mumbai','Guwahati to Chennai']

In [6]:
import time

In [7]:
model = ConcreteModel()
model.y = Var(range(9), domain = Binary)
model.obj = Objective(expr = sum(model.y[i]*c[i] for i in range(9)), sense = minimize)

model.constraints = ConstraintList()
for i in range(14):
  model.constraints.add(expr = sum(A[i][j]*model.y[j] for j in range(9)) >= 1)
time2_1 = time.time()
result = SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model)
time2_1 = time.time() - time2_1

In [8]:
model.pprint()

1 Var Declarations
    y : Size=9, Index={0, 1, 2, 3, 4, 5, 6, 7, 8}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   0.0 :     1 : False : False : Binary
          1 :     0 :   1.0 :     1 : False : False : Binary
          2 :     0 :   1.0 :     1 : False : False : Binary
          3 :     0 :   0.0 :     1 : False : False : Binary
          4 :     0 :   1.0 :     1 : False : False : Binary
          5 :     0 :   1.0 :     1 : False : False : Binary
          6 :     0 :   0.0 :     1 : False : False : Binary
          7 :     0 :   0.0 :     1 : False : False : Binary
          8 :     0 :   0.0 :     1 : False : False : Binary

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 45*y[0] + 45*y[1] + 45*y[2] + 45*y[3] + 35*y[4] + 55*y[5] + 55*y[6] + 60*y[7] + 60*y[8]

1 Constraint Declarations
    constraints : Size=14, Index={1, 2, 3, 4, 5, 6, 7, 8,

In [9]:
result.solver.termination_condition

<TerminationCondition.optimal: 'optimal'>

In [10]:
result.solver.status

<SolverStatus.ok: 'ok'>

In [11]:
model.obj()

180.0

In [14]:
#values of variables
try:
  print('Decision Variables for our model are ')
  print("||||||||||||||||||||||||||||||||||")
  for i in range(9):
    print(f'y{i} : ', model.y[i].value)
except:
  print("Problem has no Optimal.")

Decision Variables for our model are 
||||||||||||||||||||||||||||||||||
y0 :  0.0
y1 :  1.0
y2 :  1.0
y3 :  0.0
y4 :  1.0
y5 :  1.0
y6 :  0.0
y7 :  0.0
y8 :  0.0



Select crew members B, C, E, F. Total cost: 180.


In [15]:
BCEF= np.array([[0, 0, 1, 0],
                [1, 0, 0, 1],
                [0, 1, 0, 0],
                [0, 0, 0, 1],
                [1, 0, 0, 0],
                [0, 1, 0, 0],
                [0, 1, 0, 0],
                [0, 0, 0, 1],
                [1, 1, 0, 0],
                [0, 0, 0, 1],
                [0, 0, 1, 0],
                [1, 0, 0, 0],
                [0, 0, 1, 0],
                [0, 0, 0, 1]])

print(np.sum(BCEF, axis = 1))

[1 2 1 1 1 1 1 1 2 1 1 1 1 1]


We see that, flights 2, and 9 are covered two times by two crews

In [20]:
crew = ['B', 'C', 'E', 'F']
for j in range(4):
  print(f'Crew {crew[j]} assigned:', end = ' ')
  for i in range(14):
    if BCEF[i,j]>0:
      print(f'{tours[i]}', end = ',')

Crew B assigned: Mumbai to Guwahati,Delhi to Chennai,Chennai to Mumbai,Guwahati to Delhi,Crew C assigned: Mumbai to Delhi,Delhi to Kolkata,Kolkata to Chennai,Chennai to Mumbai,Crew E assigned: Mumbai to Chennai,Chennai to Guwahati,Guwahati to Mumbai,Crew F assigned: Mumbai to Guwahati,Delhi to Mumbai,Kolkata to Delhi,Chennai to Kolkata,Guwahati to Chennai,

2. Formulate it as a weighted set cover problem and solve it using a greedy algorithm. You may modify the code
used in the last lab. Pseudocode for greedy algorithm is:


In this flight representation, each crew's assignment corresponds to a subset comprising flights covered by that crew. For instance, crew A's subset is {0, 3, 7, 9}, indicating flights 0, 3, 7, and 9 are assigned to crew A. Similarly, each crew's subset represents the flights they are responsible for, ensuring comprehensive coverage of all flights collectively across all crews. This formulation facilitates efficient organization and allocation of flight assignments among multiple crews to achieve optimal operational management and scheduling

In [21]:
S = []
for i in range(9):
  s = []
  for j in range(14):
    if A[j,i]>0:
      s.append(j)
  S.append(set(s))

U = set([i for i in range(14)])

In [24]:
def greedy(S, U):
  I = set()
  index_store = []
  j = 0
  start_time = time.time()
  while I != U:
    # to find the max intersection element set
    min_element_set_index = 0
    min_cost = 1e10
    for i in range(len(S)):
      if len(S[i].difference(I)) > 0:
        cost = c[i]/len(S[i].difference(I))
        if cost < min_cost:
          min_element_set_index = i
          min_cost = cost
    #to add those elements in the set
    index_store.append(min_element_set_index)
    for element in S[min_element_set_index]:
      I.add(element)
    print("I after this step  will be ", I)
  time_taken = time.time() - start_time
  return index_store, time_taken

In [25]:
index_store, time_taken = greedy(S,U)

I after this step  will be  {1, 3, 7, 9, 13}
I after this step  will be  {1, 2, 3, 5, 6, 7, 9, 10, 12, 13}
I after this step  will be  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13}
I after this step  will be  {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13}


In [27]:
print("Index store",index_store)

Index store [5, 6, 1, 4]


It shows that crew {1, 4, 5 ,6} need to be chosen which are {B, E, F, G}

In [28]:
time2_2 = time_taken

In [30]:
cost = 0
for i in index_store:
  cost += c[i]
print("Cost for this subset selected is", cost)

Cost for this subset selected is 190


3. Suppose the flights from Chennai to Guwahati are cancelled and hence associated tours are no longer feasible.
Find the new solution using:

(a) IP formulation.

(b) Greedy Algorithm.

**NOTE**: **If flights from Chennai to Guwahati are cancelled, then all the crews that had a sequence of this flight will also be removed. Hence, the crew { D, E, G, I } will not operate. Meaning, {3,4,6,8} columns will be removed.*The cancellation of flights from Chennai to Guwahati necessitates the removal of crews that included this sequence. Consequently, crew {D, E, G, I}, corresponding to columns {3, 4, 6, 8}, will be eliminated. This adjustment ensures that crews scheduled for flights impacted by the cancellation are appropriately adjusted, maintaining operational efficiency and alignment with the revised flight schedule. Removal of these crews mitigates scheduling conflicts and ensures that operational resources are optimally allocated to accommodate the updated flight arrangements, minimizing disruptions and maximizing operational effectiveness

In [32]:
print("Tours are :\n",tours)

Tours are :
 ['Mumbai to Chennai', 'Mumbai to Guwahati', 'Mumbai to Delhi', 'Delhi to Mumbai', 'Delhi to Chennai', 'Delhi to Kolkata', 'Kolkata to Chennai', 'Kolkata to Delhi', 'Chennai to Mumbai', 'Chennai to Kolkata', 'Chennai to Guwahati', 'Guwahati to Delhi', 'Guwahati to Mumbai', 'Guwahati to Chennai']


removing chennai to guwahati row in A

In [33]:
A = np.array([[1, 0, 0, 0, 0],
              [0, 1, 0, 1, 1],
              [0, 0, 1, 0, 0],
              [1, 0, 0, 1, 1],
              [0, 1, 0, 0, 0],
              [0, 0, 1, 0, 0],
              [0, 0, 1, 0, 0],
              [1, 0, 0, 1, 1],
              [0, 1, 1, 0, 0],
              [1, 0, 0, 1, 1],
              [0, 1, 0, 0, 0],
              [0, 0, 0, 0, 0],
              [0, 0, 0, 1, 1]])

c = np.array([45,45,45,45,35,55,55,60,60])

After removing rows corresponding to cancelled flights, the last but one row, representing the Guwahati to Mumbai flight sequence, becomes devoid of scheduled flights, with all entries as 0. Consequently, this row is also removed to maintain data consistency and eliminate redundant information

In [34]:
A = np.array([[1, 0, 0, 0, 0],
              [0, 1, 0, 1, 1],
              [0, 0, 1, 0, 0],
              [1, 0, 0, 1, 1],
              [0, 1, 0, 0, 0],
              [0, 0, 1, 0, 0],
              [0, 0, 1, 0, 0],
              [1, 0, 0, 1, 1],
              [0, 1, 1, 0, 0],
              [1, 0, 0, 1, 1],
              [0, 1, 0, 0, 0],
              [0, 0, 0, 1, 1]])

c = np.array([45,45,45,55,60])

new_tour = ['Mumbai to Chennai','Mumbai to Guwahati','Mumbai to Delhi','Delhi to Mumbai',
            'Delhi to Chennai','Delhi to Kolkata','Kolkata to Chennai','Kolkata to Delhi',
            'Chennai to Mumbai','Chennai to Kolkata','Guwahati to Delhi','Guwahati to Chennai']
new_crew_set = ['A', 'B', 'C', 'F', 'H']


In [35]:
model = ConcreteModel()
model.y = Var(range(5), domain = Binary)
model.obj = Objective(expr = sum(model.y[i]*c[i] for i in range(5)), sense = minimize)

model.constraints = ConstraintList()
for i in range(12):
  model.constraints.add(expr = sum(A[i][j]*model.y[j] for j in range(5)) >= 1)
time2_3_1 = time.time()
result = SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model)
time2_3_1 = time.time() - time2_3_1

In [36]:
model.pprint()

1 Var Declarations
    y : Size=5, Index={0, 1, 2, 3, 4}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   1.0 :     1 : False : False : Binary
          1 :     0 :   1.0 :     1 : False : False : Binary
          2 :     0 :   1.0 :     1 : False : False : Binary
          3 :     0 :   1.0 :     1 : False : False : Binary
          4 :     0 :   0.0 :     1 : False : False : Binary

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 45*y[0] + 45*y[1] + 45*y[2] + 55*y[3] + 60*y[4]

1 Constraint Declarations
    constraints : Size=12, Index={1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}, Active=True
        Key : Lower : Body                                     : Upper : Active
          1 :   1.0 : y[0] + 0*y[1] + 0*y[2] + 0*y[3] + 0*y[4] :  +Inf :   True
          2 :   1.0 :     0*y[0] + y[1] + 0*y[2] + y[3] + y[4] :  +Inf :   True
          3 :   1.0 : 0*y[0

In [37]:
result.solver.termination_condition

<TerminationCondition.optimal: 'optimal'>

In [38]:
result.solver.status

<SolverStatus.ok: 'ok'>

In [39]:
model.obj()

190.0

In [40]:
#values of variables
try:
  print('Decision Variables for our model : ')
  print("|||||||||||||||||||||||||||||||||||")
  for i in range(5):
    print(f'y{i} : ', model.y[i].value)
except:
  print("Problem has  Other than Optimal.")

Decision Variables for our model : 
|||||||||||||||||||||||||||||||||||
y0 :  1.0
y1 :  1.0
y2 :  1.0
y3 :  1.0
y4 :  0.0


In [41]:
new_crew_set

['A', 'B', 'C', 'F', 'H']

This tells us that, we need to take crew  0,1,2,3:
Meaning, crew , A, B,C,F. And the total cost will be 190.
If we observe just the columns of B,C,E,F:


In [43]:
time2_3_1

0.022129535675048828

In [44]:
ABCF= np.array([[1, 0, 0, 0, 0],
              [0, 1, 0, 1, 1],
              [0, 0, 1, 0, 0],
              [1, 0, 0, 1, 1],
              [0, 1, 0, 0, 0],
              [0, 0, 1, 0, 0],
              [0, 0, 1, 0, 0],
              [1, 0, 0, 1, 1],
              [0, 1, 1, 0, 0],
              [1, 0, 0, 1, 1],
              [0, 1, 0, 0, 0],
              [0, 0, 0, 1, 1]])

print(np.sum(ABCF, axis = 1))

[1 3 1 3 1 1 1 3 2 3 1 2]


I represent flights as {0,2,3....11\}, then our problem becomes to pick certain subsets such that their union is the universal set fo flights. Here the subsets will be the set of flights that are covered by each crew.

In [46]:
#creating subset from incidence matrix
S = []
for i in range(5):
  s = []
  for j in range(12):
    if A[j,i]>0:
      s.append(j)
  S.append(set(s))

U = set([i for i in range(12)])

S

[{0, 3, 7, 9}, {1, 4, 8, 10}, {2, 5, 6, 8}, {1, 3, 7, 9, 11}, {1, 3, 7, 9, 11}]

In [47]:
index_store, time2_3_2 = greedy(S,U)

I after this step  will be  {1, 3, 7, 9, 11}
I after this step  will be  {1, 2, 3, 5, 6, 7, 8, 9, 11}
I after this step  will be  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}
I after this step  will be  {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}


In [48]:
index_store

[3, 2, 1, 0]

It shows that crew {0,1,2,3} will be chosen which are **A, B,C,F**

In [49]:
time2_3_2

0.0038275718688964844

In [50]:
cost = 0
for i in index_store:
  cost += c[i]
print("Cost for this subset selected is: ", cost)

Cost for this subset selected is:  190


4. Compare the solutions of 2.1, 2.2 and 2.3 given by the integer program and greedy algorithm in the following
counts:

(a) Running time.

(b) Ratio of objective value of IP solution to Greedy solution.

In [51]:
print("time2_1 is",time2_1)
print("time2_2 is",time2_2)
print("time2_3_1 is", time2_3_1)
print("time2_3_2 is", time2_3_2)

time2_1 :  0.04083847999572754
time2_2 :  0.00026106834411621094
time2_3_1:  0.022129535675048828
time2_3_2:  0.0038275718688964844



In general, Greedy algorithms often execute faster than Integer Linear Programming (ILP) because they prioritize immediate gains without exhaustive search, unlike ILP, which aims for the most optimal solution, resulting in longer computation times.

In [55]:
obj = [180, 190, 190, 190]

In [56]:
print("Ratio of ILP to Greedy for part(1) to part(2) is", obj[0]/obj[1])
print("Ratio of ILP to Greedy for part(2) to part(3) is ", obj[2]/obj[3])

Ratio of ILP to Greedy for part(1) to part(2) is 0.9473684210526315
Ratio of ILP to Greedy for part(2) to part(3) is  1.0


In Problem 2.1, where the objective is to minimize cost, ILP outperforms Greedy by providing a cost of 180. However, in Problem 2.3, where maximizing value is the goal and crew size is reduced, both ILP and Greedy perform similarly. This is attributed to fewer feasible solutions due to the reduced crew size, limiting the advantage of ILP's exhaustive search for the most optimal solution. Hence, in scenarios with constrained options, the performance gap between ILP and Greedy narrows, resulting in comparable outcomes.